In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# URLs crudas de los archivos CSV en el repositorio de GitHub
train_url = "https://raw.githubusercontent.com/gustavoleon-py/hBridgeDatasets/main/train_chb_data.csv"
eval_url = "https://raw.githubusercontent.com/gustavoleon-py/hBridgeDatasets/main/eval_chb_data.csv"

# Cargar datasets desde GitHub
try:
    train_df = pd.read_csv(train_url)
    eval_df = pd.read_csv(eval_url)
except Exception as e:
    print(f"Error al cargar los archivos desde GitHub: {e}")
    print("Asegúrate de que los archivos 'train_chb_data.csv' y 'eval_chb_data.csv' estén en el repositorio.")
    exit()

# Preparar datos de entrenamiento
# Entradas: condiciones + lambda_i + lambda_v + lambda_sw (10 características)
X_train = train_df.drop(columns=["THD", "switching_loss"]).values
y_train = train_df[["THD", "switching_loss"]].values

# Escalar datos para mejor entrenamiento
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

# Crear el modelo surrogate (predice THD y switching_loss)
model = Sequential([
    Dense(128, activation='relu', input_shape=(10,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2)  # Salidas: THD y switching_loss
])

# Función de pérdida personalizada para entrenamiento: MSE con peso mayor en THD (2.0)
def custom_cost_loss(y_true, y_pred):
    error_thd = y_true[:, 0] - y_pred[:, 0]
    weighted_error_thd = 2.0 * tf.square(error_thd)  # Mayor peso en THD

    error_loss = y_true[:, 1] - y_pred[:, 1]
    error_loss_sq = tf.square(error_loss)

    # Regularización: penaliza predicciones negativas (THD y switching_loss >=0)
    reg_thd = tf.reduce_mean(tf.maximum(0.0, -y_pred[:, 0]))
    reg_loss = tf.reduce_mean(tf.maximum(0.0, -y_pred[:, 1]))

    mse = tf.reduce_mean(weighted_error_thd + error_loss_sq)
    reg = 0.1 * (reg_thd + reg_loss)

    return mse + reg

# Compilar y entrenar
model.compile(optimizer='adam', loss=custom_cost_loss)
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Función para optimizar lambdas para condiciones fijas
def optimize_lambdas(fixed_conditions, model, scaler_X, scaler_y, learning_rate=0.01, steps=200):
    lambdas = tf.Variable([2.5, 2.5, 2.5], dtype=tf.float32)  # Inicializar lambda_i, lambda_v, lambda_sw
    optimizer = tf.optimizers.Adam(learning_rate)

    for _ in range(steps):
        with tf.GradientTape() as tape:
            # Construir input completo: condiciones + lambdas
            input_full = tf.concat([fixed_conditions, lambdas], axis=0)
            # Ensure scaling is done within the tape or with tf operations
            # Convert scaled data back to tensor before passing to the model
            input_scaled = scaler_X.transform(input_full.numpy().reshape(1, -1))
            input_tensor = tf.convert_to_tensor(input_scaled, dtype=tf.float32)

            # Predecir THD y switching_loss
            pred_scaled = model(input_tensor)
            pred = scaler_y.inverse_transform(pred_scaled.numpy())
            pred_thd, pred_switching_loss = pred[0]

            # Costo multivariable: 2 * THD + switching_loss (refleja prioridad en THD)
            # Ensure operations on predictions are with tensors if needed for gradients
            pred_thd_tensor = tf.convert_to_tensor(pred_thd, dtype=tf.float32)
            pred_switching_loss_tensor = tf.convert_to_tensor(pred_switching_loss, dtype=tf.float32)
            cost = 2.0 * pred_thd_tensor + pred_switching_loss_tensor

        # Gradientes respecto a lambdas
        grads = tape.gradient(cost, lambdas)
        optimizer.apply_gradients(zip([grads], [lambdas]))

        # Clamp lambdas a [0, 5]
        lambdas.assign(tf.clip_by_value(lambdas, 0.0, 5.0))

    return lambdas.numpy(), cost.numpy() # Return cost as numpy as well

# Optimizar para 10 ejemplos aleatorios del dataset de evaluación
eval_sample = eval_df.sample(10, random_state=42)
for idx, row in eval_sample.iterrows():
    fixed_conditions = row[["num_cells", "Vdc", "C_dc", "fsw", "R_load", "L_load", "mod_index"]].values
    fixed_conditions_tf = tf.convert_to_tensor(fixed_conditions, dtype=tf.float32)

    opt_lambdas, opt_cost = optimize_lambdas(fixed_conditions_tf, model, scaler_X, scaler_y)
    original_thd = row["THD"]
    original_switching_loss = row["switching_loss"]
    original_cost = 2.0 * original_thd + original_switching_loss

    print(f"Ejemplo {idx}:")
    print(f"  Condiciones: {fixed_conditions}")
    print(f"  Lambdas originales: λ_i={row['lambda_i']:.3f}, λ_v={row['lambda_v']:.3f}, λ_sw={row['lambda_sw']:.3f} | Costo original: {original_cost:.3f}")
    print(f"  Lambdas optimizados: λ_i={opt_lambdas[0]:.3f}, λ_v={opt_lambdas[1]:.3f}, λ_sw={opt_lambdas[2]:.3f} | Costo optimizado: {opt_cost:.3f}\n")

# Guardar el modelo para uso futuro
model.save("chb_surrogate_model.h5")
print("Modelo guardado como 'chb_surrogate_model.h5'")